In [ ]:
import sys as _sys
import os

currentPath = os.path.abspath(os.getcwd())

split = currentPath.split("Cshells")
if len(split)<2:
    print("Please rename the repository 'Cshells'")
    raise ValueError
pathToPythonScripts = split[0] + "Cshells/python/"
pathToModels = os.path.join(split[0], "Cshells/data/models")
pathToOutputs = os.path.join(split[0], "Cshells/output")

_sys.path.insert(0, pathToPythonScripts)

In [ ]:
import MeshFEM
import ElasticRods

import average_angle_linkages
from bending_validation import suppress_stdout as so
import elastic_rods
import math
import numpy as np
import pickle
import torch

from CShell import CShell
from CurvesDoFOptimizer import CurvesDoFOptimizer
from linkage_vis import LinkageViewer, LinkageViewerWithSurface
import mesh
from open_average_angle_linkage import ComputeStressesHistogram, open_average_angle_linkage, RunAndAnalyzeDeployment
import py_newton_optimizer
from vis.fields import ScalarField
from VisUtils import ConvergencePlotsVisualizer, PlotStackedConvergencePlots
from VisUtilsDeployment import CompareDeploymentStatistics

torch.set_default_dtype(torch.float64)
    
def ToNumpy(tensor):
    return tensor.cpu().detach().clone().numpy()

PI = math.pi

# Initialization

You can pick any example that has the requested files located in `data/models/`.

In [ ]:
modelName = "dome01"

with open(os.path.join(pathToModels, modelName, "flat_optimized.p"), 'rb') as f:
# with open(os.path.join(pathToModels, modelName, "flat_initial.p"), 'rb') as f:
    flatLinkage = pickle.load(f)
    
with open(os.path.join(pathToModels, modelName, "deployed_optimized.p"), 'rb') as f:
# with open(os.path.join(pathToModels, modelName, "deployed_initial.p"), 'rb') as f:
    depLinkage = pickle.load(f)
    
linkagesGuess = {
    "flat": flatLinkage,
    "deployed": depLinkage,
}

In [ ]:
with open(os.path.join(pathToModels, modelName, "cshell_optimized.p"), 'rb') as f:
# with open(os.path.join(pathToModels, modelName, "cshell_initial.p"), 'rb') as f:
    dictCShell = pickle.load(f)
    
cshell = CShell(
    dictCShell["curvesDoF"], dictCShell["nJ"], dictCShell["curves"], dictCShell["curvesFamily"], dictCShell["nCPperRodEdge"], 
    dictCShell["alphaTar"], dictCShell["mult"], dictCShell["subdivision"], symmetry=dictCShell["symmetry"],
    attractionMesh=dictCShell["attractionMesh"], targetMesh=dictCShell["targetMesh"],
    rodMaterial=dictCShell["flatLinkage"].homogenousMaterial(), optimizeAlpha=True, useSAL=True, 
    linkagesGuess=linkagesGuess, flatOnly=False,
)

In [ ]:
cshell.flatView.show()

In [ ]:
cshell.deployedView.show()

In [ ]:
# with open("cshell_initial.p", 'wb') as f:
#     pickle.dump(cshell.GetCShellParams(), f)

# with open("flat_initial.p", 'wb') as f:
#     pickle.dump(cshell.flatLinkage, f)
    
# with open("deployed_initial.p", 'wb') as f:
#     pickle.dump(cshell.deployedLinkage, f)

# Deployment

Although the model has already been deployed in the saved file, we show how one can redeploy the model.

In [ ]:
numOpeningSteps = 30
maxNewtonIterIntermediate = 500 # maximum number of equilibrium steps taken per opening step

## As an average angle linkage

First, we copy the flat linkage, which we call `depLink` since it will be deployed later. The linkage is set as an `AverageAngleLinkage` and has 6 degrees of freedom removed to discard rigid transformations during deployment. We pick the most central joint to speed up each intermediate equilibrium step during deployment.

In [ ]:
depLink = average_angle_linkages.AverageAngleLinkage(cshell.flatLinkage)
driver = cshell.flatLinkage.centralJoint()
jdo = cshell.flatLinkage.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6))
depView = LinkageViewer(depLink, width=768, height=480)

depView.show()

Then, we define the optimizer for the intermediate equilibrium steps during deployment. For sanity check, we run equilibrium in the flat layout. As you can see, the flat layout is already at rest: the beams's rest shape are chosen so that the drawn curves are at rest in the flat state.

In [ ]:
newtonOptimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
newtonOptimizerOptions.gradTol = 1.0e-7
newtonOptimizerOptions.verbose = 1
newtonOptimizerOptions.beta = 1.0e-8
newtonOptimizerOptions.niter = 500
newtonOptimizerOptions.verboseNonPosDef = False

def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.gradTol = 1.0e-5
    return average_angle_linkages.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

report = average_angle_linkages.compute_equilibrium(
    depLink, elastic_rods.TARGET_ANGLE_NONE, options=newtonOptimizerOptions, fixedVars=fixedVars,
)

The linkage is now incrementally opened by the increment `alphaGap`. You can see the linkage deploying in the viewer above.

In [ ]:
alphaGap = cshell.deployedLinkage.getAverageActuatedJointsAngle() - depLink.getAverageActuatedJointsAngle()

with so(): 
    open_average_angle_linkage(
        depLink, driver, alphaGap, numOpeningSteps, 
        depView, equilibriumSolver=equilibriumSolver, 
        maxNewtonIterationsIntermediate=maxNewtonIterIntermediate
    )

## As an average angle surface attracted linkage (AASAL)

We repeat the steps above, this time specifying the linkage as an `AverageAngleSurfaceAttractedLinkage`.

In [ ]:
depLinkAASAL = average_angle_linkages.AverageAngleSurfaceAttractedLinkage(
    cshell.attractionMesh["V"], cshell.attractionMesh["F"], 
    False, cshell.flatLinkage
)

depLinkAASAL.attraction_weight = 1.0e-5
depLinkAASAL.scaleJointWeights(jointPosWeight=0.5)
depLinkAASAL.set_holdClosestPointsFixed(False)
depLinkAASAL.setTargetSurface(cshell.attractionMesh["V"], cshell.attractionMesh["F"])
depLinkAASAL.setTargetJointsPosition(cshell.attractionMesh["targetJP"].reshape(-1,))

attractionSurf = mesh.Mesh(*(cshell.attractionMesh["V"], cshell.attractionMesh["F"]))
depViewAASAL = LinkageViewerWithSurface(depLinkAASAL, attractionSurf, wireframeSurf=False, transparent=True, width=768, height=480)

depViewAASAL.show()

Running the initial equilibrium solve does align the target surface and the linkage. Note that there are no `fixedVars` needed this time since rigid transformation are ruled out thanks to the attraction surface.

In [ ]:
newtonOptimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
newtonOptimizerOptions.gradTol = 1.0e-7
newtonOptimizerOptions.verbose = 1
newtonOptimizerOptions.beta = 1.0e-8
newtonOptimizerOptions.niter = 500
newtonOptimizerOptions.verboseNonPosDef = False

def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.gradTol = 1.0e-5
    return average_angle_linkages.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

with so():
    report = average_angle_linkages.compute_equilibrium(
        depLinkAASAL, elastic_rods.TARGET_ANGLE_NONE, options=newtonOptimizerOptions, fixedVars=[],
    )

depViewAASAL.update()

And the final deployment using the target surface as reference.

In [ ]:
alphaGap = cshell.deployedLinkage.getAverageActuatedJointsAngle() - depLinkAASAL.getAverageActuatedJointsAngle()

with so(): 
    open_average_angle_linkage(
        depLinkAASAL, driver, alphaGap, numOpeningSteps, 
        depViewAASAL, equilibriumSolver=equilibriumSolver, 
        maxNewtonIterationsIntermediate=maxNewtonIterIntermediate
    )